# Imports

In [1]:
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm 
import statistics
import torch
import random
import time
import numpy as np
from transformers import AdamW
from torch.utils.tensorboard import SummaryWriter
import pickle
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
import shutil

from GRU_model import *
from config import *

# Seeds for Comparisons:

In [2]:
torch.manual_seed(1)
np.random.seed(2)
random.seed(3)

# Read Data

In [3]:
with open('/Users/mvilenko/Desktop/CPI_HRNN - version 2.0/mayas_project/SGRU/data/train_dataset.pickle', 'rb') as f:
    train_dataset = pickle.load(f)
    
with open('/Users/mvilenko/Desktop/CPI_HRNN - version 2.0/mayas_project/SGRU/data/test_dataset.pickle', 'rb') as f:
    test_dataset = pickle.load(f)

In [4]:
train_dataset.shape

(17828, 14)

In [14]:
test_dataset

,Inflation t-12,Inflation t-11,Inflation t-10,Inflation t-9,Inflation t-8,Inflation t-7,Inflation t-6,Inflation t-5,Inflation t-4,Inflation t-3,Inflation t-2,Inflation t-1,Inflation t,Inflation t+1
51379,0.346759,0.122860,-0.287493,-0.254331,-0.265488,0.124384,0.387922,0.256546,0.419889,-0.161491,-0.142182,0.241591,-0.007559,0.307399
51380,0.307399,0.346759,0.122860,-0.287493,-0.254331,-0.265488,0.124384,0.387922,0.256546,0.419889,-0.161491,-0.142182,0.241591,0.461293
51381,0.461293,0.307399,0.346759,0.122860,-0.287493,-0.254331,-0.265488,0.124384,0.387922,0.256546,0.419889,-0.161491,-0.142182,0.266383
51382,0.266383,0.461293,0.307399,0.346759,0.122860,-0.287493,-0.254331,-0.265488,0.124384,0.387922,0.256546,0.419889,-0.161491,0.757317
51383,0.757317,0.266383,0.461293,0.307399,0.346759,0.122860,-0.287493,-0.254331,-0.265488,0.124384,0.387922,0.256546,0.419889,0.157281
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37173,-0.132909,0.176656,1.067445,-0.762470,-0.225467,-0.130396,0.258277,0.408885,0.522500,0.956554,0.263887,0.526196,0.282573,0.484400
37174,0.484400,-0.132909,0.176656,1.067445,-0.762470,-0.225467,-0.130396,0.258277,0.408885,0.522500,0.956554,0.263887,0.526196,0.820072
37175,0.820072,0.484400,-0.132909,0.176656,1.067445,-0.762470,-0.225467,-0.130396,0.258277,0.408885,0.522500,0.956554,0.263887,0.019381
37176,0.019381,0.820072,0.484400,-0.132909,0.176656,1.067445,-0.762470,-0.225467,-0.130396,0.258277,0.408885,0.522500,0.956554,-0.651313


# Create Dataloader

In [6]:
def create_dataloader(category_train_df, category_test_df):
    x_train = category_train_df.iloc[:,:-1].to_numpy()
    y_train = category_train_df.iloc[:,-1].to_numpy()
    x_test = category_test_df.iloc[:,:-1].to_numpy()
    y_test= category_test_df.iloc[:,-1].to_numpy()

    x_train= torch.from_numpy(x_train).to(torch.float32)
    y_train = torch.from_numpy(y_train).to(torch.float32)
    x_test = torch.from_numpy(x_test).to(torch.float32)
    y_test = torch.from_numpy(y_test).to(torch.float32)

    train_dataset = TensorDataset(x_train, y_train)
    test_dataset = TensorDataset(x_test, y_test)
    train_dataloader =  DataLoader(train_dataset, batch_size=BatchSize, shuffle=False)
    test_dataloader =  DataLoader(test_dataset, batch_size=BatchSize, shuffle=False)
    return train_dataloader, test_dataloader

# Model Configurations

In [7]:
#Define our device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

# Training Loop

In [8]:
def training_loop(model, train_dataloader, optimizer):
    running_loss = 0
    model.train()
    predictions_list = []
    for inputs, labels in train_dataloader:
        # initialize calculated gradients (from prev step)
        optimizer.zero_grad()
        inputs, labels = inputs.to(device), labels.to(device)
        #Changing input shape - last batch size can change so we define it as input.shape[0]
        inputs = inputs.view(inputs.shape[0], SequenceLength, Features) 
        #model prediction
        pred = model(inputs)
        #append batch predictions to predictions list
        predictions_list.append(pred.view(1,-1))
        # calculate loss
        loss = Criterion(pred, labels.view(-1,1))
        # calculate the gradient
        loss.backward()
        # update parameters
        optimizer.step()
        #Add to loss of batch to epoch train loss
        running_loss+=loss.item()
    # Calculte the epoch train loss
    epoch_train_loss = running_loss/len(train_dataloader.dataset)
    return epoch_train_loss


In [9]:
def evaluation_loop(model, test_dataloader):
     # Evaluation
    # Initiate test loss, accuracy and f1 score to zero
    test_loss = 0
    # Change model to eval mode
    model.eval()
    # we dont need to update weights, so we define no_grad() to save memory

    predictions_list = []
    with torch.no_grad():
        for inputs, labels in test_dataloader:
            inputs = inputs.view(inputs.shape[0], SequenceLength, Features)
            inputs, labels = inputs.to(device), labels.to(device)
            out = model(inputs)
            predictions_list.append(out.view(1,-1))
            test_batch_loss = Criterion(out, labels.view(-1,1))
            test_loss += test_batch_loss.item()
    # Calculate epoch loss
    epoch_predictions = torch.cat(predictions_list, dim=1)
    epoch_test_loss = test_loss/len(test_dataloader.dataset)
        
    return epoch_test_loss, epoch_predictions


In [10]:
def save_checkpoint(checkpoint, is_best, checkpoint_path, best_model_path):
    """
    checkpoint: checkpoint we want to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    # save checkpoint data to the path given, checkpoint_path
    torch.save(checkpoint, checkpoint_path)
    # if it is a best model, min validation loss
    if is_best:
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(checkpoint_path, best_model_path)

In [11]:
def load_checkpoint(checkpoint_path, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into       
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_path)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # return model, optimizer, epoch value, min validation loss 
    return model, optimizer, checkpoint['epoch'], valid_loss_min

In [12]:
def training_and_evaluation(model, train_dataloader, test_dataloader, optim, checkpoint_path, best_checkpoint_path):
   #results list
   train_loss_list = []
   test_loss_list = []

   #Create writer for using tesndorboard
   writer = SummaryWriter(log_dir=f'{TbDirectory}')

   min_test_loss = np.inf

   for epoch in range(Epochs):
      #initiate train epoch loss
      epoch_train_loss = training_loop(model, train_dataloader, optim)
      epoch_test_loss, epoch_test_predictions = evaluation_loop(model, test_dataloader)

      checkpoint = {
         'epoch': epoch + 1,
         'valid_loss_min': epoch_test_loss,
         'state_dict': model.state_dict(),
         'optimizer': optim.state_dict(),
        }
      
      # save checkpoint
      save_checkpoint(checkpoint, False, checkpoint_path, best_checkpoint_path)

      if epoch_test_loss <= min_test_loss:
         save_checkpoint(checkpoint, True, checkpoint_path, best_checkpoint_path)
         min_test_loss = epoch_test_loss

      train_loss_list.append(epoch_train_loss)
      test_loss_list.append(epoch_test_loss)

      # Display those measures on tensorboard
      writer.add_scalar(tag='loss/train', scalar_value=epoch_train_loss, global_step=epoch)
      writer.add_scalar(tag='loss/test', scalar_value=epoch_test_loss, global_step=epoch)
    
   results = {'train_loss': train_loss_list, 'test_loss': test_loss_list}
   return results

In [13]:
def pipline(train_dataset, test_dataset):
    train_dataloader, test_dataloader = create_dataloader(train_dataset, test_dataset)

    model = GRUModel(input_dim = Features, hidden_dim = HiddenSize, layer_dim = LayersDim, output_dim = OutputDim, dropout_prob = DropoutProb)
    model.to(device)
    
    optimizer = torch.optim.AdamW(model.parameters(), lr=Lr)

    parameters_file_name = 'single_gru_params.pt'
    
    results = training_and_evaluation(
                            model=model,
                            optim=optimizer,
                            train_dataloader=train_dataloader,
                            test_dataloader=test_dataloader,
                            checkpoint_path=CheckpointPath+parameters_file_name,
                            best_checkpoint_path=BestcheckpointPath+parameters_file_name,
                        )


    return results

In [15]:
results = pipline(train_dataset, test_dataset)

In [16]:
with open('/Users/mvilenko/Desktop/CPI_HRNN - version 2.0/mayas_project/SGRU/data/model_results.pickle', 'wb') as handle:
    pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Create Best Model

In [17]:
def create_best_model(dir_path):
    basic_model = GRUModel(input_dim = Features, hidden_dim = HiddenSize, layer_dim = LayersDim, output_dim = OutputDim, dropout_prob = DropoutProb)
    basic_optimizer = torch.optim.AdamW(basic_model.parameters(), lr=Lr)
    basic_model.to(device)

    ckp_path = dir_path+'single_gru_params.pt'
    best_model, optimizer, checkpoint, valid_loss_min = load_checkpoint(ckp_path, basic_model, basic_optimizer)
        
    return best_model

In [18]:
dir_path = "checkpoints/best_checkpoints/"

best_model = create_best_model(dir_path)

In [19]:
best_model

GRUModel(
  (gru): GRU(1, 64, batch_first=True)
  (fc): Linear(in_features=64, out_features=1, bias=True)
)

# Get Best Predictions for Each Category 

In [20]:
def get_best_predictions(best_model):
    train_dataloader, test_dataloader = create_dataloader(train_dataset, test_dataset)
    epoch_test_loss, epoch_predictions = evaluation_loop(best_model, test_dataloader)
    best_predictions = epoch_predictions

    return best_predictions

In [ ]:
best_predictions = get_best_predictions(best_model)

In [ ]:
with open('/Users/mvilenko/Desktop/CPI_HRNN - version 2.0/mayas_project/SGRU/data/predictions.pickle', 'wb') as handle:
    pickle.dump(best_predictions, handle, protocol=pickle.HIGHEST_PROTOCOL)